In [35]:
import numpy as np 
import pandas as pd 
import random
from collections import Counter
from typing import Tuple
from typing import List
import sys

In [2]:
class colors():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['♠', '♦', '♥', '♣'] #['S','H','D','C'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [3]:
class numbers():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [4]:
class card():
    __slots__ = '__color', '__number'
    
    def __init__(self,aColor, aNumber):
        
        self.__color = aColor
        self.__number = aNumber
        
    def getColor(self):
        return self.__color
    
    def getNumber(self):
        return self.__number
    

In [5]:
class hand():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard:card):
        self.__cards.append(aCard)
        self.__numbersOnly.append(aCard.getNumber())
        self.__colorsOnly.append(aCard.getColor())
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly
    
    def visualizeHand(self):
        myVisu = []
        for card in self.__cards:
            myVisu.append(card.getNumber() + card.getColor())
        print(myVisu)
            

        

In [6]:
class deck():
    __slots__ = '__cards'
        
    def __init__(self):
        self.__cards = []
        for color in colors().getPossibilities():
            for number in numbers().getPossibilities():
                self.__cards.append(card(color, number))
        random.shuffle(self.__cards)
    
    def drawCard(self):
        return self.__cards.pop(0)
    
    def getDeck(self):
        return self.__cards

In [7]:
class river():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard:card):
        self.__cards.append(aCard)
        self.__numbersOnly.append(aCard.getNumber())
        self.__colorsOnly.append(aCard.getColor())
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly
    
    def visualizeHand(self):
        myVisu = []
        for card in self.__cards:
            myVisu.append(card.getNumber() + card.getColor())
        print(myVisu)
            


In [8]:
def isFlush(aHand:hand) -> bool:
    return max(Counter(aHand.getColors()).values()) >= 5

def isStraight(aHand:hand) -> Tuple[bool, int]:
    first = True
    countConseq = 0
    maxConSeq = 0
    lastCardSeq = 0
    myIndexedSortedHand = sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])
    for idx in myIndexedSortedHand:
        if not first:
            if idx == myLastIdx+1:
                countConseq = countConseq+1
                lastCardSeq = myLastIdx #to compare between straights
            else:
                countConseq = 0
            if countConseq > maxConSeq :
                maxConSeq = countConseq
        first = False
        myLastIdx = idx
    
    return maxConSeq>=5,int(lastCardSeq)
    
    
def howManySameNumber(aHand:hand)-> Tuple[List,List,List]:
    myCount = Counter(aHand.getNumbers())
    
#     nbFourOfAKind = Counter(myCount.values())[4] #how many four of a kind ..
    cards4 = [numbers().getPossibilities().index(x) for x in [k for (k, v) in myCount.items() if v == 4]]
#     nbThreeOfAKind = Counter(myCount.values())[3]
    cards3 = [numbers().getPossibilities().index(x) for x in [k for (k, v) in myCount.items() if v == 3]]
#     nbPair = Counter(myCount.values())[2]
    cards2 = [numbers().getPossibilities().index(x) for x in [k for (k, v) in myCount.items() if v == 2]]
    
    return cards2,cards3,cards4
    
def singleCard(aHand:hand, idxBest=1): #idxBest starts at 1 for best, 2 for snd Best
    
    return sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])[-idxBest] #get number with highest

In [9]:
def getValue(aHand:hand) -> float:
    myValue = 0
    myStraight, myidx = isStraight(aHand)
    myFlush = isFlush(aHand)
    myRoyal = myidx is len(numbers().getPossibilities())-1
    
    myPairs, myThrees, myFour = howManySameNumber(aHand) 
    
    myValue = myValue + myStraight*myFlush*myRoyal*200200000000 + myStraight*myFlush*10200000000*myidx
   
    if len(myFour):
        myFourValue = 560000000*myFour[0]
    else:
        myFourValue = 0
    
    if len(myThrees)==2 or (len(myThrees)==1 and len(myPairs)):
        myFullHouse= 28000000 * max(myThrees)
    else:
        myFullHouse = 0
    
    
    if len(myPairs)==2:
        myTwoPairs = 70000 * sorted(myPairs)[1]+4000 * sorted(myPairs)[0]
    else:
        myTwoPairs = 0
    
    if len(myThrees)==1:
        myThreeCard = 1400000 * max(myThrees)
    else:
        myThreeCard = 0
        
    if len(myPairs)==1:
        myOnePair = 200 * max(myPairs)
    else:
        myOnePair = 0
    
    myValue = myValue + myFourValue + myFullHouse + 40000*myFlush + 2000*myStraight*myidx
    myValue = myValue + myThreeCard + myTwoPairs + myOnePair 
    
    initCoef = 10
    for i in range(0,7):
        myValue = myValue + singleCard(aHand,i+1)*initCoef
        initCoef = initCoef/20
    
    return myValue
    

In [10]:
a = ['K', 'J', 'A','A'] 
possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A']

sorted(np.unique(a), key=lambda x: possibilities.index(x))

['J', 'K', 'A']

In [31]:
nbPlayers = 20
myDeck = deck()
mySmallHands = [] #individualHand
myBigHands = [] #Hand with River
for i in range(0,nbPlayers):
    mySmallHands.append(hand())
    myBigHands.append(hand())
myRiver = river()

In [32]:

#draw cards
for drawnCard in range(0,2):
    for player in range(0,nbPlayers):
        myCard = myDeck.drawCard()
        mySmallHands[player].addCard(myCard)
        myBigHands[player].addCard(myCard)
        
#draw cards in the river
for drawnCard in range(0,5):
    myCard = myDeck.drawCard()
    
    myRiver.addCard(myCard)
    for player in range(0,nbPlayers):
        myBigHands[player].addCard(myCard)

In [33]:
for i in range(0,nbPlayers):
    print(getValue(myBigHands[i]))

794115.7328874998
770115.7615
806115.7618249999
2315.7315062499997
786115.7340125
323400115.7865
786115.7302593751
2315.7301937499997
2315.730128125
806115.7366374999
2315.731509375
2315.734003125
2315.7301937499997
15400115.786700001
2315.735325
802115.7355124999
2315.761503125
2315.732825
2315.734003125
2315.7617625000003


In [34]:
for i in range(0,nbPlayers):
    print(myBigHands[i].visualizeHand())

['8♣', '8♥', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['K♦', '2♠', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['Q♦', 'K♣', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['7♠', '4♠', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['9♣', '6♠', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['A♥', '2♦', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['6♣', '5♣', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['4♣', '5♥', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['4♦', '3♣', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['Q♣', '8♦', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['7♣', '5♦', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['3♠', '9♥', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['5♠', '4♥', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['A♠', '9♠', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['J♦', '7♦', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['J♠', 'J♥', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['K♠', '3♥', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['8♠', '7♥', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['9♦', '3♦', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None
['J♣', 'K♥', 'Q♥', 'A♦', '6♦', 'A♣', '2♣']
None


['J♣', '3♣', 'A♥', '7♠', '7♥', '2♥', '5♥']


ValueError: <class 'int'> is not in list

In [ ]:
class game():
    
    __slots__ = "nbPlayers", "deck", "smallHands", "bigHands", "river", "judgerEnabler", 'winner'
    
    def __init__(self, nbOfPlayer : int):
        self.judgerEnabler = 0
        self.nbPlayers = nbOfPlayer
        self.deck = deck()
        self.smallHands = [] #individualHand
        self.bigHands = [] #Hand with River
        
        for i in range(0,nbPlayers):
            self.smallHands.append(hand())
            self.bigHands.append(hand())
            
        self.river = river()
        self.winner = None
        
    def dealCards() -> "Nothing":
        #draw cards
        if self.judgerEnabler == 0:
            for drawnCard in range(0,2):
                for player in range(0,self.nbPlayers):
                    myCard = myDeck.drawCard()
                    self.smallHands[player].addCard(myCard)
                    self.bigHands[player].addCard(myCard)
        else:
            sys.stderr("Already dealt cards to players")
        self.judgerEnabler = 1
                
    def dealFlop() -> "Nothing":
        if self.judgerEnabler == 1:
            for drawnCard in range(0,3):
                myCard = self.deck.drawCard()

                self.river.addCard(myCard)
                for player in range(0,self.nbPlayers):
                    self.bigHands[player].addCard(myCard)
        elif self.judgerEnabler == 0:
            sys.stderr("Not dealt cards to players yet, please do")
        elif self.judgerEnabler > 1:
            sys.stderr("Already did the flop")
        self.judgerEnabler == 2:
            
    def dealTurn() -> "Nothing":
        if self.judgerEnabler == 2:
            for drawnCard in range(0,1):
                myCard = self.deck.drawCard()

                self.river.addCard(myCard)
                for player in range(0,self.nbPlayers):
                    self.bigHands[player].addCard(myCard)
        elif self.judgerEnabler == 0:
            sys.stderr("Not dealt cards to players yet, please do")
        elif self.judgerEnabler == 1:
            sys.stderr("Flop has not been done, please do things in order")
        elif self.judgerEnabler > 2:
            sys.stderr("Already did the Turn, it is now time for the River")
        self.judgerEnabler == 3:
        
    def dealRiver() -> "Nothing":
        if self.judgerEnabler == 3:
            for drawnCard in range(0,1):
                myCard = self.deck.drawCard()

                self.river.addCard(myCard)
                for player in range(0,self.nbPlayers):
                    self.bigHands[player].addCard(myCard)
            _judgeTheGame()
            
        elif self.judgerEnabler == 0:
            sys.stderr("Not dealt cards to players yet, please do")
        elif self.judgerEnabler == 1:
            sys.stderr("Flop has not been done, please do things in order")
        elif self.judgerEnabler == 2:
            sys.stderr("Turn has not been done, please do things in order")
        elif self.judgerEnabler > 3:
            sys.stderr("the Game is Over")
        self.judgerEnabler == 4:
        
    def _judge():
        myValues = []
        for i in range(0,self.nbPlayers):
            myValues.append(getValue(self.bigHands[i]))
        myValues.index(max(myValues))